In [1]:
import pandas as pd
import glob
import os

# --- Configuração ---
# Altere esta variável para o caminho da sua pasta 'data'
caminho_para_pasta = './data/' # '.' significa o diretório atual

In [2]:

from dataload import carregar_dataframe_particionado


dfMarcacao = carregar_dataframe_particionado(caminho_para_pasta, 'marcacao')


--- Iniciando carregamento de tabelas particionadas ---
DataFrame 'marcacao' carregado com 3,122,088 linhas a partir de 57 arquivos.
DataFrame 'oferta_programada' carregado com 9,200,972 linhas a partir de 24 arquivos.
DataFrame 'profissional_historico' carregado com 5,064,858 linhas a partir de 18 arquivos.
DataFrame 'solicitacao' carregado com 3,210,746 linhas a partir de 40 arquivos.

--- Iniciando carregamento de tabelas únicas ---
DataFrame 'cids' carregado com 12,454 linhas.
DataFrame 'equipamento_historico' carregado com 194,228 linhas.
DataFrame 'habilitacao_historico' carregado com 21,166 linhas.
DataFrame 'leito_historico' carregado com 27,795 linhas.
DataFrame 'procedimento' carregado com 806 linhas.
DataFrame 'tempo_espera' carregado com 70,104 linhas.
DataFrame 'unidade_historico' carregado com 25,026 linhas.

--- Carregamento Concluído! ---

Exemplo: 5 primeiras linhas do DataFrame 'marcacao':
                         profissional_solicitante_id  \
0  a3d4f399a0cce984fdfe

In [ ]:
dfMarcacao.head()

In [ ]:
condicao_absenteismo = (dfMarcacao['marcacao_executada'] == False) | (dfMarcacao['falta_registrada'] == True)
dfMarcacao['alvo_absenteismo'] = condicao_absenteismo.astype(int)

# --- Verificação ---
print("Criação da variável alvo concluída!")
print(f"Total de agendamentos relevantes para análise: {len(dfMarcacao):,}")

print("\nDistribuição da variável alvo (alvo_absenteismo):")
# value_counts() mostra quantos 0s (Compareceu) e 1s (Faltou) temos.
print(dfMarcacao['alvo_absenteismo'].value_counts())

print("\nExemplo de linhas do DataFrame final:")
# Mostra a nova coluna junto com as colunas que a originaram
print(dfMarcacao[['data_marcacao', 'solicitacao_status', 'marcacao_executada', 'falta_registrada', 'alvo_absenteismo']].head(10))

In [ ]:
import numpy as np
dfMarcacao['data_solicitacao'] = pd.to_datetime(dfMarcacao['data_solicitacao'], errors='coerce')
dfMarcacao['data_marcacao'] = pd.to_datetime(dfMarcacao['data_marcacao'], errors='coerce')


In [ ]:
# 1. Lead Time (Tempo de Espera em dias)
dfMarcacao['lead_time_dias'] = (dfMarcacao['data_marcacao'] - dfMarcacao['data_solicitacao']).dt.days
# Tratar casos onde a data de solicitação pode ser posterior à marcação (dados sujos)
dfMarcacao['lead_time_dias'] = dfMarcacao['lead_time_dias'].clip(lower=0)


In [ ]:
# 2. Dia da Semana (0=Segunda, 1=Terça, ..., 6=Domingo)
dfMarcacao['dia_semana'] = dfMarcacao['data_marcacao'].dt.dayofweek

# 3. Hora do Dia
dfMarcacao['hora_dia'] = dfMarcacao['data_marcacao'].dt.hour

# 4. Mês
dfMarcacao['mes'] = dfMarcacao['data_marcacao'].dt.month

print("Atributos temporais criados: 'lead_time_dias', 'dia_semana', 'hora_dia', 'mes'")


In [ ]:
# Primeiro, ordenamos o dataframe por paciente e data. É CRUCIAL.
dfMarcacao = dfMarcacao.sort_values(by=['paciente_id', 'data_marcacao'])


In [ ]:
# Calculamos o número de agendamentos e faltas anteriores
# groupby().cumsum() calcula a soma cumulativa dentro de cada grupo (paciente)
# .shift(1) desloca os dados para que cada linha veja o total ANTES de si mesma
grouped = dfMarcacao.groupby('paciente_id')
dfMarcacao['num_agendamentos_anteriores'] = grouped.cumcount()
dfMarcacao['num_faltas_anteriores'] = grouped['alvo_absenteismo'].cumsum().shift(1).fillna(0)


In [ ]:
# 2. Taxa de Absenteísmo Histórica
# Usamos np.where para evitar divisão por zero para pacientes em seu primeiro agendamento
dfMarcacao['taxa_absenteismo_anterior'] = np.where(
    dfMarcacao['num_agendamentos_anteriores'] > 0,
    dfMarcacao['num_faltas_anteriores'] / dfMarcacao['num_agendamentos_anteriores'],
    0  # Se não há agendamentos anteriores, a taxa é 0
)


In [ ]:
# --- C. Fatores Geográficos e de Distância ---
print("\n--- Iniciando Engenharia de Atributos Geográficos (C) ---")
dfUnidadeHistorico = carregar_dataframe_particionado(caminho_para_pasta, 'unidade_historico')
# Supondo que dfUnidadeHistorico está carregado
# Como a tabela de unidades é histórica, vamos pegar a informação mais recente para cada CNES
unidades = dfUnidadeHistorico.sort_values(['ano', 'mes'], ascending=False).drop_duplicates('unidade_id_cnes')


In [ ]:
# 1. Join para obter dados da unidade SOLICITANTE
dfMarcacao = pd.merge(
    dfMarcacao,
    unidades[['unidade_id_cnes', 'unidade_bairro', 'unidade_latitude', 'unidade_longitude']],
    left_on='unidade_solicitante_id_cnes',
    right_on='unidade_id_cnes',
    how='left',
    suffixes=('', '_solicitante')
)

In [ ]:
# 2. Join para obter dados da unidade EXECUTANTE
dfMarcacao = pd.merge(
    dfMarcacao,
    unidades[['unidade_id_cnes', 'unidade_bairro', 'unidade_latitude', 'unidade_longitude']],
    left_on='unidade_executante_id_cnes',
    right_on='unidade_id_cnes',
    how='left',
    suffixes=('_solicitante', '_executante')
)

In [ ]:
# 3. Função para calcular a Distância de Haversine (distância em linha reta na Terra)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em km
    
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance

In [ ]:
# 4. Feature de Distância
dfMarcacao['distancia_km'] = haversine_distance(
    dfMarcacao['unidade_latitude_solicitante'],
    dfMarcacao['unidade_longitude_solicitante'],
    dfMarcacao['unidade_latitude_executante'],
    dfMarcacao['unidade_longitude_executante']
)

# 5. Feature Mesmo Bairro
dfMarcacao['mesmo_bairro'] = (dfMarcacao['unidade_bairro_solicitante'] == dfMarcacao['unidade_bairro_executante']).astype(int)

print("Atributos geográficos criados: 'distancia_km', 'mesmo_bairro'")

In [ ]:
# --- D. Fatores Clínicos e do Procedimento ---
print("\n--- Iniciando Engenharia de Atributos Clínicos (D) ---")
# Risco (solicitacao_risco) e Tipo de Vaga (vaga_consumida_tp) já existem.

dfProcedimento = carregar_dataframe_particionado(caminho_para_pasta, 'procedimento')
dfCids = carregar_dataframe_particionado(caminho_para_pasta, 'cids')



In [ ]:
dfCids.head()

In [ ]:
# Supondo que dfProcedimento e dfCids estão carregados
# 1. Join com a tabela de Procedimentos
dfMarcacao = pd.merge(
    dfMarcacao,
    dfProcedimento[['procedimento_sisreg_id', 'procedimento_especialidade']],
    on='procedimento_sisreg_id',
    how='left'
)

# 2. Join com a tabela de CID
dfMarcacao = pd.merge(
    dfMarcacao,
    dfCids[['cid_id', 'cid']], # Pegando a categoria do CID
    left_on='cid_agendado_id',
    right_on='cid_id',
    how='left'
)

print("Atributos clínicos adicionados: 'procedimento_especialidade', 'cid_categoria_descricao'")


In [ ]:
# --- Limpeza Final ---
# Remover colunas de ID e geográficas que não são mais necessárias para o modelo
colunas_para_remover = [
    'unidade_id_cnes_solicitante', 'unidade_bairro_solicitante', 'unidade_latitude_solicitante', 'unidade_longitude_solicitante',
    'unidade_id_cnes_executante', 'unidade_bairro_executante', 'unidade_latitude_executante', 'unidade_longitude_executante',
    'cid_id'
]
dfMarcacao.drop(columns=colunas_para_remover, inplace=True, errors='ignore')


In [ ]:
# Verificando as novas colunas
novas_colunas = [
    'lead_time_dias', 'dia_semana', 'hora_dia', 'mes',
    'num_agendamentos_anteriores', 'taxa_absenteismo_anterior',
    'distancia_km', 'mesmo_bairro',
    'procedimento_especialidade', 'cid', 'alvo_absenteismo'
]
print("\nAmostra do DataFrame final com as novas features:")
print(dfMarcacao[novas_colunas].head())

print("\nInformações sobre as novas colunas (verificar nulos):")
print(dfMarcacao[novas_colunas].info())

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, log_loss


In [ ]:
# Supondo que 'df_modelagem' é o DataFrame final da etapa de Engenharia de Atributos
# Certifique-se de que a coluna de data de marcação está no formato datetime
dfMarcacao['data_marcacao'] = pd.to_datetime(dfMarcacao['data_marcacao'])

In [ ]:
# Selecionar as features (X) e o alvo (y)
# Vamos excluir colunas de ID, datas e texto livre que não serão usadas diretamente
features_para_usar = [
    'lead_time_dias', 'dia_semana', 'hora_dia', 'mes', 
    'num_agendamentos_anteriores', 'taxa_absenteismo_anterior',
    'distancia_km', 'mesmo_bairro', 'paciente_sexo', 'paciente_faixa_etaria',
    'paciente_avisado', 'solicitacao_risco', 'vaga_consumida_tp',
    'procedimento_especialidade', 'cid'
]
alvo = 'alvo_absenteismo'

X = dfMarcacao[features_para_usar]
y = dfMarcacao[alvo]


In [ ]:
# Identificar colunas numéricas e categóricas automaticamente
numerical_features = X.select_dtypes(include=['int64', 'float64', 'int32']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print("--- Variáveis Definidas ---")
print(f"Features numéricas: {numerical_features}")
print(f"Features categóricas: {categorical_features}")


In [ ]:
# --- Passo 2: Divisão Temporal dos Dados ---

# Ordenar os dados por data para garantir uma divisão temporal correta
df_modelagem_sorted = dfMarcacao.sort_values('data_marcacao')

# Definir um ponto de corte (ex: 80% dos dados para treino, 20% para teste)
split_point = int(len(df_modelagem_sorted) * 0.8)
train_df = df_modelagem_sorted.iloc[:split_point]
test_df = df_modelagem_sorted.iloc[split_point:]

X_train = train_df[features_para_usar]
y_train = train_df[alvo]
X_test = test_df[features_para_usar]
y_test = test_df[alvo]

print(f"\n--- Divisão Temporal ---")
print(f"Dados de treino: {len(X_train):,} linhas, de {train_df['data_marcacao'].min():%Y-%m-%d} a {train_df['data_marcacao'].max():%Y-%m-%d}")
print(f"Dados de teste: {len(X_test):,} linhas, de {test_df['data_marcacao'].min():%Y-%m-%d} a {test_df['data_marcacao'].max():%Y-%m-%d}")

In [ ]:
# Criar um pipeline de pré-processamento para o modelo
# Para features numéricas: preencher valores nulos com a mediana e escalar
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Para features categóricas: preencher nulos com 'missing' e aplicar One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
# Juntar os pré-processadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Criar o pipeline final que inclui o pré-processamento e o modelo
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', LogisticRegression(random_state=42, solver='liblinear'))])


In [ ]:
# --- Código de Diagnóstico ---
print("\n--- Verificando a distribuição do alvo nos conjuntos ---")
print("Distribuição em y_train:")
print(y_train.value_counts())

print("\nDistribuição em y_test:")
print(y_test.value_counts())

In [ ]:
# --- NOVO DIAGNÓSTICO: Investigar as colunas-fonte em dfMarcacao ---

print("--- Investigando as colunas originais em dfMarcacao ---")

print("\nValores únicos e contagem na coluna 'marcacao_executada':")
# O dropna=False nos mostrará se existem valores nulos (NaN)
print(dfMarcacao['marcacao_executada'].value_counts(dropna=False))

print("\n----------------------------------------------------")

print("\nValores únicos e contagem na coluna 'falta_registrada':")
print(dfMarcacao['falta_registrada'].value_counts(dropna=False))

print("\n----------------------------------------------------")

print("\nValores únicos e contagem na coluna 'solicitacao_status' (Top 15):")
# Esta coluna é nossa melhor candidata para encontrar o status de falta
print(dfMarcacao['solicitacao_status'].value_counts(dropna=False).head(15))